In [8]:
import numpy as np
import math
import matplotlib.pyplot as plt  
from sklearn.metrics import mean_squared_error
import MySQLdb

from keras.layers import Dropout
from keras.layers import Dense,Input
from keras.models import Sequential,Model
from keras.callbacks import EarlyStopping
from sklearn.svm import SVR

import sys
import xlwt
##get data##
import MySQLdb
from sklearn.preprocessing import MinMaxScaler


def transfromData(trainRaw, testRaw,windosSize):  ##Train ratial, train, test
    sc = MinMaxScaler(feature_range = (0, 1))
    #npRaw = np.array(rawData)
    #scaledData = sc.fit_transform(npRaw.reshape(-1,1))  
    #trainData = np.array(scaledData[:trainCount]).reshape(-1,1)##train 
    #testData = np.array(scaledData[-testCount:]).reshape(-1,1)
    
    npTrain = sc.fit_transform(np.array(trainRaw).reshape(-1,1))
    npTest = sc.fit_transform(np.array(npTest).reshape(-1,1))
    

    X_train, y_train = splitXy(npTrain,windosSize)
    X_test, y_test = splitXy(npTest,windosSize)
    return sc, X_train, y_train, X_test, y_test

def splitXy(data,windosSize):
    windows = []
    for i in range(windosSize, data.shape[0]):
        windows.append(data[i-windosSize:i, 0])
    np.random.shuffle(windows)
    X = []
    y = []
    for i in range(len(windows)):
        X.append(windows[i][:windosSize-1])
        y.append(windows[i][-1:])
    X, y = np.array(X), np.array(y)
    #X = np.reshape(X, (X.shape[0], X.shape[1], 1))
    return X,y
def buildModel():
    dnn_1_input = Input(shape=(6,),name='dnn_1_input')
    dnn_1 = Dense(128,activation="tanh", name='dnn_1')(dnn_1_input)
    dnn_2 = Dense(64,activation="tanh", name='dnn_2')(dnn_1)
    dnn_3 = Dense(32,activation="tanh", name='dnn_3')(dnn_2)
    dnn_4 = Dense(1, activation='tanh', name='dnn_4')(dnn_3)

    model = Model(dnn_1_input,dnn_4)
    model.summary()
    model.compile(loss='mse',
                  optimizer='adam')
    return model
def getStationList(cursor):
    stationList = []
    queryString = "SELECT distinct station from cleandata"
    cursor.execute(queryString)
    results = cursor.fetchall()
    for row in results :
        val = str(row).strip("',()")
        stationList.append(val)
    stationList.remove('富貴角')##only 106少數資料
    return stationList
def Visualize():
    predicted = sc.inverse_transform(regressor.predict(X_test))
    originY = sc.inverse_transform (y_test)
    print("MSE : ["+str(mean_squared_error(predicted, originY))+"]")
    # Visualising the results
    plt.plot(originY[:100], color = 'red', label = 'Real')  
    plt.plot(predicted[:100], color = 'blue', label = 'Predicted ') 
    plt.legend()
    plt.show()
def writeExcelHead(sheet1,epochs,station):
    sheet1.write(0,1,station)
    raw = 1
    for e in range(epochs):
        sheet1.write(raw,0,e+1)
        raw+=1
def fetchData(cursor,station,windosSize):
    queryString = "SELECT * from cleandata where station='"+station+"' and item = 'PM2.5' and date NOT like '2017%'"
    cursor.execute(queryString)
    results = cursor.fetchall()

    trainRawData = []
    for row in results:
        for i in range(3,27):
            trainRawData.append(float(row[i]))
            
    queryString = "SELECT * from cleandata where station='"+station+"' and item = 'PM2.5' and date like '2017%'"
    cursor.execute(queryString)
    results = cursor.fetchall()

    testRawData = []
    for row in results:
        for i in range(3,27):
            testRawData.append(float(row[i]))

    sc, X_train, y_train, X_test, y_test = transfromData(trainRawData,testRawData,windosSize)
    return sc, X_train, y_train, X_test, y_test
def trainSVR(sc, X_train, y_train, X_test, y_test):
    clf = SVR(gamma='scale',kernel ='linear', epsilon=0.02)
    clf.fit(X_train, y_train.ravel())
    predicted_SVR = sc.inverse_transform(clf.predict(X_test).reshape(-1,1))
    ori_y_test = sc.inverse_transform(y_test)
    mse = mean_squared_error(predicted_SVR, ori_y_test)
    print(mse)
    return mse

Using TensorFlow backend.


tf.estimator package not installed.
tf.estimator package not installed.


In [22]:
import datetime
db=MySQLdb.connect(host="localhost",user="root", passwd="swater0", db="airdb", charset="utf8")
cursor = db.cursor()

stationList = getStationList(cursor)
col=1
for station in stationList:
    print("training : " +station)
    mseList = []
    for windowSize in range(2,22):
        sc, X_train, y_train, X_test, y_test = fetchData(cursor,station,windowSize)
        mse = trainSVR(sc, X_train, y_train, X_test, y_test)
        mseList.append(mse)
    book = xlwt.Workbook(encoding="utf-8")
    sheet1 = book.add_sheet("Sheet1")
    writeExcelHead(sheet1,20,station)
    row = 1
    for mse in mseList:
        sheet1.write(row,1,mse)
        row+=1
    book.save("excelFiles/SVR/SVRresult"+station+".xls")
db.close()


training : 二林
47.326101431427375
45.79922088004736
45.24678228334987
41.3956855488899
41.03391758976141
41.03207060700602
40.21926113324242
40.1871660942331
40.18069617605827
40.048480846059874
40.043131531690314
40.049633537140814
40.00674220818088
40.000279099568814
39.99400857477838
39.931792547489046
39.91843315304148
39.916554427140085
39.80849684910707
39.800302202827616
training : 南投
23.632376187390616
20.491820617399004
19.37224064719802
19.179033434858454
19.0849160904624
19.08236194225283
19.0291352243965
19.030249974737806
19.00612519499636
18.995259069022904
18.982281615760726
18.97443122244862
18.968761483179083
18.96524803149038
18.93831098630523
18.934760200295504
18.924193588225112
18.909699957926758
18.88667425696549
18.868807927240155
training : 埔里
28.281764139385395
24.975624866188912
23.488434518282897
23.214096958596805
22.92210015560395
22.89162491907019
22.840165282283802
22.82355088135301
22.811726916371963
22.78876535084733
22.774263832073487
22.75751690793992


12.461548059457666
12.434385853106606
12.42964600129285
12.419449941426512
12.41924654080333
12.418881669760527
12.41705438406319
12.406080729261449
12.398881643715463
12.395910437074235
12.389886991450847
12.375910174733015
12.369466018103935
12.366239618142137
12.366993293239798
training : 新莊
21.059039267824733
18.871041733180597
18.049691785224557
17.888022033462665
17.79437507806328
17.79395350184812
17.782072446678708
17.778495973733598
17.775955390007834
17.77481132286607
17.772691209487775
17.77375112356685
17.7730169726036
17.773261875212572
17.772011969561078
17.761810930572285
17.76519967805545
17.755156241809654
17.754058755431142
17.75310655273874
training : 松山
28.149461331927547
28.1887322243348
27.033230207770163
25.11122923764712
25.000675783301876
24.7898196500164
24.359943825079796
24.339409030085722
24.298165878581003
24.176289377298595
24.17269584079166
24.170419915414772
24.101468660630655
24.09894477514992
24.089047829391216
24.057394203116292
24.04908863584198
24.

34.11703895773856
34.075805380764514
34.07265915583624
34.06099886849982
34.06283075196993
34.03838807152428
34.03919622214305
34.04216764135334
34.035692094056074
33.867729213815245
33.831625243059484
training : 頭份
22.013034383911272
19.64568536978828
18.329703272719215
18.234579667879302
17.909362922809475
17.89064232019379
17.84838089893544
17.84902973828305
17.80581710732972
17.78718624936083
17.776639839286283
17.765100374403684
17.761770153619967
17.76174559637729
17.759542870197375
17.73807701929786
17.731340416337403
17.731927212432787
17.731926183239473
17.724537972642366
training : 臺東
7.7407421355712716
7.607836970041346
6.913977587164627
6.8079480943012545
6.673377989481283
6.651682730422685
6.5946196056080035
6.589080794529679
6.569994768573007
6.569294580558651
6.561533918550902
6.56100507579982
6.559203124843931
6.5596704096639575
6.55571656641872
6.555589446343644
6.5556584274963186
6.554503843952051
6.550038523520384
6.549036860304174
training : 花蓮
11.844523061882198
11

40.756521756578316
training : 大寮
53.70381687615551
51.69901000269764
50.55075762415191
50.55685321858411
50.52799562443455
50.48950950449853
50.287701376525334
50.26396324766995
50.22660068879075
50.15979639889003
49.95152569412476
49.917480289288534
49.92716737210282
49.87255870782834
49.75260244937869
49.68823654311767
49.61717748280854
49.417083115396764
49.15837124893141
49.06550731953878
training : 小港
54.144855436128054
53.91884540150785
53.37040381605136
50.0464414766813
49.99281928158193
49.99735902804676
49.345146216756
49.332040809165036
49.31160678703235
49.08904108115518
49.087436008646286
49.05340255577454
49.0243375621644
49.010387710212655
48.98793861502936
48.92239430604181
48.857971220402675
48.75387371464385
48.65895994606768
48.53051453204743
training : 屏東
47.28461522243319
46.67886259561108
46.23988799467849
42.046207491927326
41.87160559978077
41.90580058291517
40.886453785286264
40.878883568151394
40.87947672676228
40.57449943664531
40.575179857194726
40.5733953514